In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn

%matplotlib inline

On-Base Percentage (OBP) = Hits + Walks(BB) + Hit By Pitch(HBP) / At Bats(AB) + Walks + HBP + Sacrifice Flies

Note = Sacrifice Flies stats don't start until 1954.

Before that, SH = SF + Bunts

In [2]:
column_id = [0, 13, 14]
bio = pd.read_csv('data/Master.csv', usecols=column_id)
len(bio)

18846

In [3]:
bio = bio.rename(columns={'nameFirst': 'FirstName', 'nameLast': 'LastName'})

In [4]:
bio.head()

,playerID,FirstName,LastName
0,aardsda01,David,Aardsma
1,aaronha01,Hank,Aaron
2,aaronto01,Tommie,Aaron
3,aasedo01,Don,Aase
4,abadan01,Andy,Abad


In [5]:
column_id = [0, 1, 3, 4]
salary = pd.read_csv('data/Salaries.csv', usecols=column_id)
len(salary)

25575

In [6]:
column_id = [0, 1, 3, 5]
fielding = pd.read_csv('data/Fielding.csv', usecols=column_id)
len(fielding)

170526

In [7]:
column_id = [0, 1, 3, 5, 6, 8, 15, 18, 20]
batting = pd.read_csv('data/Batting.csv', usecols=column_id)
len(batting)

101332

In [8]:
data = pd.merge(batting, bio)
data = pd.merge(data, fielding)
data = pd.merge(data, salary)

In [9]:
len(data)

46885

In [10]:
data.head()

,playerID,yearID,teamID,G,AB,H,BB,HBP,SF,FirstName,LastName,POS,salary
0,johnto01,1985,OAK,11,NaN,NaN,NaN,NaN,NaN,Tommy,John,P,600000
1,johnto01,1986,NYA,13,NaN,NaN,NaN,NaN,NaN,Tommy,John,P,60000
2,johnto01,1987,NYA,33,NaN,NaN,NaN,NaN,NaN,Tommy,John,P,350000
3,johnto01,1988,NYA,35,NaN,NaN,NaN,NaN,NaN,Tommy,John,P,375000
4,rosepe01,1985,CIN,119,405.0,107.0,86.0,4.0,4.0,Pete,Rose,1B,358858


In [11]:
data_dropna = data.copy()

In [12]:
data_dropna = data_dropna.dropna()

In [13]:
len(data_dropna)

45008

In [14]:
data_dropna = data_dropna.drop_duplicates()

In [15]:
len(data_dropna)

44969

In [16]:
data_dropna['OBP'] = (data_dropna['H']+data_dropna['BB']+data_dropna['HBP'])/(data_dropna['AB']+data_dropna['BB']+data_dropna['HBP']+data_dropna['SF'])

In [17]:
data_dropna.head()

,playerID,yearID,teamID,G,AB,H,BB,HBP,SF,FirstName,LastName,POS,salary,OBP
4,rosepe01,1985,CIN,119,405.0,107.0,86.0,4.0,4.0,Pete,Rose,1B,358858,0.394790
5,rosepe01,1986,CIN,72,237.0,52.0,30.0,4.0,1.0,Pete,Rose,1B,1000000,0.316176
6,staubru01,1985,NYN,54,45.0,12.0,10.0,0.0,0.0,Rusty,Staub,OF,300000,0.400000
7,staubru01,1985,NYN,54,45.0,12.0,10.0,0.0,0.0,Rusty,Staub,RF,300000,0.400000
10,perezto01,1985,CIN,72,183.0,60.0,22.0,0.0,2.0,Tony,Perez,1B,225000,0.396135


In [18]:
data_dropna['Ratio'] = data_dropna['OBP']/data_dropna['salary']

In [19]:
moneyball = data_dropna[data_dropna['AB'] > 30]

In [20]:
moneyball.sort_values('Ratio', ascending=False).head()

,playerID,yearID,teamID,G,AB,H,BB,HBP,SF,FirstName,LastName,POS,salary,OBP,Ratio
6095,jamesdi01,1993,NYA,115,343.0,114.0,31.0,2.0,1.0,Dion,James,LF,0,0.38992,inf
6097,jamesdi01,1993,NYA,115,343.0,114.0,31.0,2.0,1.0,Dion,James,RF,0,0.38992,inf
6092,jamesdi01,1993,NYA,115,343.0,114.0,31.0,2.0,1.0,Dion,James,1B,0,0.38992,inf
6093,jamesdi01,1993,NYA,115,343.0,114.0,31.0,2.0,1.0,Dion,James,CF,0,0.38992,inf
6094,jamesdi01,1993,NYA,115,343.0,114.0,31.0,2.0,1.0,Dion,James,DH,0,0.38992,inf


In [21]:
moneyball.head()

,playerID,yearID,teamID,G,AB,H,BB,HBP,SF,FirstName,LastName,POS,salary,OBP,Ratio
4,rosepe01,1985,CIN,119,405.0,107.0,86.0,4.0,4.0,Pete,Rose,1B,358858,0.394790,1.100128e-06
5,rosepe01,1986,CIN,72,237.0,52.0,30.0,4.0,1.0,Pete,Rose,1B,1000000,0.316176,3.161765e-07
6,staubru01,1985,NYN,54,45.0,12.0,10.0,0.0,0.0,Rusty,Staub,OF,300000,0.400000,1.333333e-06
7,staubru01,1985,NYN,54,45.0,12.0,10.0,0.0,0.0,Rusty,Staub,RF,300000,0.400000,1.333333e-06
10,perezto01,1985,CIN,72,183.0,60.0,22.0,0.0,2.0,Tony,Perez,1B,225000,0.396135,1.760601e-06


In [22]:
moneyball.describe()

,yearID,G,AB,H,BB,HBP,SF,salary,OBP,Ratio
count,35674.000000,35674.000000,35674.000000,35674.000000,35674.000000,35674.000000,35674.000000,3.567400e+04,35674.000000,3.567400e+04
mean,1999.619443,95.327045,292.966530,77.698324,28.793071,2.580283,2.495291,1.850219e+06,0.314491,inf
std,8.684972,41.900731,178.285269,52.923830,23.630328,3.197577,2.413448,3.159003e+06,0.057569,NaN
min,1985.000000,8.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,1992.000000,61.000000,135.000000,32.000000,10.000000,0.000000,1.000000,2.700000e+05,0.288462,1.618589e-07
50%,1999.000000,98.000000,268.000000,68.000000,23.000000,2.000000,2.000000,5.250000e+05,0.321196,5.508307e-07
75%,2007.000000,132.000000,444.750000,119.000000,41.000000,4.000000,4.000000,2.000000e+06,0.350711,1.136364e-06
max,2015.000000,163.000000,716.000000,262.000000,232.000000,35.000000,17.000000,3.300000e+07,0.609400,inf


In [23]:
moneyball[moneyball['playerID'] == 'carltst01']

,playerID,yearID,teamID,G,AB,H,BB,HBP,SF,FirstName,LastName,POS,salary,OBP,Ratio
13,carltst01,1986,PHI,16,34.0,7.0,1.0,0.0,0.0,Steve,Carlton,P,1000000,0.228571,2.285714e-07


In [24]:
salary[salary['playerID'] == 'carltst01']

,yearID,teamID,playerID,salary
389,1985,PHI,carltst01,1075000
673,1986,CHA,carltst01,60000
1072,1986,PHI,carltst01,1000000
2229,1988,MIN,carltst01,115000


In [25]:
column_id = [0, 1, 3, 4]
new_sal = pd.read_csv('data/Salaries.csv', usecols=column_id)

In [26]:
new_sal[new_sal['playerID'] == 'carltst01']

,yearID,teamID,playerID,salary
389,1985,PHI,carltst01,1075000
673,1986,CHA,carltst01,60000
1072,1986,PHI,carltst01,1000000
2229,1988,MIN,carltst01,115000


In [27]:
moneyball['OBP'].describe()

count    35674.000000
mean         0.314491
std          0.057569
min          0.000000
25%          0.288462
50%          0.321196
75%          0.350711
max          0.609400
Name: OBP, dtype: float64

In [28]:
fielding[fielding['playerID'] == 'carltst01']

,playerID,yearID,teamID,POS
64332,carltst01,1965,SLN,P
65690,carltst01,1966,SLN,P
67057,carltst01,1967,SLN,P
68449,carltst01,1968,SLN,P
69864,carltst01,1969,SLN,P
71617,carltst01,1970,SLN,P
73354,carltst01,1971,SLN,P
75046,carltst01,1972,PHI,P
76725,carltst01,1973,PHI,P
78470,carltst01,1974,PHI,P


In [29]:
batting[batting['playerID'] == 'carltst01']

,playerID,yearID,teamID,G,AB,H,BB,HBP,SF
44328,carltst01,1965,SLN,15,2.0,0.0,0.0,0.0,0.0
45077,carltst01,1966,SLN,9,15.0,4.0,0.0,1.0,0.0
45855,carltst01,1967,SLN,30,72.0,11.0,3.0,0.0,0.0
46626,carltst01,1968,SLN,35,73.0,12.0,0.0,0.0,0.0
47376,carltst01,1969,SLN,32,80.0,17.0,3.0,0.0,0.0
48301,carltst01,1970,SLN,34,80.0,16.0,1.0,4.0,0.0
49220,carltst01,1971,SLN,37,96.0,17.0,4.0,0.0,0.0
50102,carltst01,1972,PHI,41,117.0,23.0,5.0,1.0,0.0
50996,carltst01,1973,PHI,40,100.0,16.0,2.0,0.0,1.0
51883,carltst01,1974,PHI,39,102.0,25.0,0.0,0.0,3.0


In [43]:
moneyball = moneyball.sort_values('Ratio', ascending=False)

In [44]:
moneyball.head(15)

,playerID,yearID,teamID,G,AB,H,BB,HBP,SF,FirstName,LastName,POS,salary,OBP,Ratio
44596,heywaja01,2010,ATL,142,520.0,144.0,91.0,10.0,2.0,Jason,Heyward,OF,400000,0.393258,9.831461e-07
44597,heywaja01,2010,ATL,142,520.0,144.0,91.0,10.0,2.0,Jason,Heyward,RF,400000,0.393258,9.831461e-07
41836,hanigry01,2010,CIN,70,203.0,61.0,33.0,4.0,2.0,Ryan,Hanigan,C,415000,0.404959,9.758040e-07
41434,bartoda02,2010,OAK,159,556.0,152.0,110.0,3.0,5.0,Daric,Barton,1B,410000,0.393175,9.589636e-07
42746,gonzaca01,2010,COL,145,587.0,197.0,40.0,2.0,7.0,Carlos,Gonzalez,LF,406000,0.375786,9.255817e-07
42748,gonzaca01,2010,COL,145,587.0,197.0,40.0,2.0,7.0,Carlos,Gonzalez,RF,406000,0.375786,9.255817e-07
42745,gonzaca01,2010,COL,145,587.0,197.0,40.0,2.0,7.0,Carlos,Gonzalez,CF,406000,0.375786,9.255817e-07
42747,gonzaca01,2010,COL,145,587.0,197.0,40.0,2.0,7.0,Carlos,Gonzalez,OF,406000,0.375786,9.255817e-07
43712,freesda01,2010,SLN,70,240.0,71.0,21.0,4.0,1.0,David,Freese,3B,400000,0.360902,9.022556e-07
43711,freesda01,2010,SLN,70,240.0,71.0,21.0,4.0,1.0,David,Freese,1B,400000,0.360902,9.022556e-07


In [37]:
moneyball_2010 = moneyball[moneyball.yearID == 2010]

In [38]:
moneyball_2010.head(15)

,playerID,yearID,teamID,G,AB,H,BB,HBP,SF,FirstName,LastName,POS,salary,OBP,Ratio
44596,heywaja01,2010,ATL,142,520.0,144.0,91.0,10.0,2.0,Jason,Heyward,OF,400000,0.393258,9.831461e-07
44597,heywaja01,2010,ATL,142,520.0,144.0,91.0,10.0,2.0,Jason,Heyward,RF,400000,0.393258,9.831461e-07
41836,hanigry01,2010,CIN,70,203.0,61.0,33.0,4.0,2.0,Ryan,Hanigan,C,415000,0.404959,9.758040e-07
41434,bartoda02,2010,OAK,159,556.0,152.0,110.0,3.0,5.0,Daric,Barton,1B,410000,0.393175,9.589636e-07
42746,gonzaca01,2010,COL,145,587.0,197.0,40.0,2.0,7.0,Carlos,Gonzalez,LF,406000,0.375786,9.255817e-07
42745,gonzaca01,2010,COL,145,587.0,197.0,40.0,2.0,7.0,Carlos,Gonzalez,CF,406000,0.375786,9.255817e-07
42747,gonzaca01,2010,COL,145,587.0,197.0,40.0,2.0,7.0,Carlos,Gonzalez,OF,406000,0.375786,9.255817e-07
42748,gonzaca01,2010,COL,145,587.0,197.0,40.0,2.0,7.0,Carlos,Gonzalez,RF,406000,0.375786,9.255817e-07
43711,freesda01,2010,SLN,70,240.0,71.0,21.0,4.0,1.0,David,Freese,1B,400000,0.360902,9.022556e-07
43712,freesda01,2010,SLN,70,240.0,71.0,21.0,4.0,1.0,David,Freese,3B,400000,0.360902,9.022556e-07


In [34]:
moneyball_2010 = moneyball_2010.dropna()
moneyball_2010 = moneyball_2010.drop_duplicates(['playerID'])
moneyball_2010 = moneyball_2010.drop_duplicates(['POS'])

In [35]:
moneyball_2010.head(15)

,playerID,yearID,teamID,G,AB,H,BB,HBP,SF,FirstName,LastName,POS,salary,OBP,Ratio
44596,heywaja01,2010,ATL,142,520.0,144.0,91.0,10.0,2.0,Jason,Heyward,OF,400000,0.393258,9.831461e-07
41836,hanigry01,2010,CIN,70,203.0,61.0,33.0,4.0,2.0,Ryan,Hanigan,C,415000,0.404959,9.758040e-07
41434,bartoda02,2010,OAK,159,556.0,152.0,110.0,3.0,5.0,Daric,Barton,1B,410000,0.393175,9.589636e-07
42746,gonzaca01,2010,COL,145,587.0,197.0,40.0,2.0,7.0,Carlos,Gonzalez,LF,406000,0.375786,9.255817e-07
40932,narvech01,2010,MIL,37,49.0,16.0,2.0,1.0,0.0,Chris,Narveson,P,406500,0.365385,8.988551e-07
42963,lowrije01,2010,BOS,55,171.0,49.0,25.0,1.0,0.0,Jed,Lowrie,SS,434000,0.380711,8.772135e-07
39247,choosh01,2010,CLE,144,550.0,165.0,83.0,11.0,2.0,Shin-Soo,Choo,RF,461100,0.400929,8.695051e-07
44019,rasmuco01,2010,SLN,144,464.0,128.0,63.0,1.0,4.0,Colby,Rasmus,CF,418000,0.360902,8.634025e-07
42636,dewitbl01,2010,LAN,82,256.0,69.0,30.0,3.0,1.0,Blake,DeWitt,2B,410000,0.351724,8.578638e-07
43814,johnsch05,2010,HOU,94,341.0,105.0,15.0,2.0,4.0,Chris,Johnson,3B,400000,0.337017,8.425414e-07


In [36]:
moneyball_2010.drop(moneyball_2010.index[0])

,playerID,yearID,teamID,G,AB,H,BB,HBP,SF,FirstName,LastName,POS,salary,OBP,Ratio
41836,hanigry01,2010,CIN,70,203.0,61.0,33.0,4.0,2.0,Ryan,Hanigan,C,415000,0.404959,9.758040e-07
41434,bartoda02,2010,OAK,159,556.0,152.0,110.0,3.0,5.0,Daric,Barton,1B,410000,0.393175,9.589636e-07
42746,gonzaca01,2010,COL,145,587.0,197.0,40.0,2.0,7.0,Carlos,Gonzalez,LF,406000,0.375786,9.255817e-07
40932,narvech01,2010,MIL,37,49.0,16.0,2.0,1.0,0.0,Chris,Narveson,P,406500,0.365385,8.988551e-07
42963,lowrije01,2010,BOS,55,171.0,49.0,25.0,1.0,0.0,Jed,Lowrie,SS,434000,0.380711,8.772135e-07
39247,choosh01,2010,CLE,144,550.0,165.0,83.0,11.0,2.0,Shin-Soo,Choo,RF,461100,0.400929,8.695051e-07
44019,rasmuco01,2010,SLN,144,464.0,128.0,63.0,1.0,4.0,Colby,Rasmus,CF,418000,0.360902,8.634025e-07
42636,dewitbl01,2010,LAN,82,256.0,69.0,30.0,3.0,1.0,Blake,DeWitt,2B,410000,0.351724,8.578638e-07
43814,johnsch05,2010,HOU,94,341.0,105.0,15.0,2.0,4.0,Chris,Johnson,3B,400000,0.337017,8.425414e-07
